# Explore USGS Terrain COGs
This 1 arc second (~30m) resolution terrain data as 32-bit cloud-optimized geotiff in a public AWS bucket.  Each file is a 1x1 degree square. 

In [ ]:
import hvplot.xarray
import fsspec
import os
import rioxarray

Use GDAL settings from [COG Best Practices](https://github.com/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb)

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

Explore a sample terrain square (the 1x1 degree square that contains most of Cape Cod)

In [ ]:
url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w071/USGS_1_n42w071.tif'

In [ ]:
da = rioxarray.open_rasterio(url, masked=True).squeeze('band')

In [ ]:
da

Mask out the ocean

In [ ]:
da = da.where(da>0)

Visualize with hvplot from the holoviz.org tool suite

In [ ]:
da.hvplot.image(x='x', y='y', geo=True, rasterize=True, 
                frame_width=500, cmap='rainbow', tiles='ESRI')

#### Explore the full list of available degree squares

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
flist = fs.glob('prd-tnm/StagedProducts/Elevation/1/TIFF/')

In [ ]:
flist[:5]

In [ ]:
flist[-5:]